In [ ]:
from __future__ import annotations
import json
from datamodel import Order, ProsperityEncoder, TradingState, Symbol, OrderDepth
import math


class Trader:
    def __init__(self):
        # Initialize position limit and position for each product
        self.products = ["PINACOLADA", "COCONUTS"]
        self.pos_limit = {product: 600 if product == "PINACOLADA" else 300 for product in self.products}
        self.pos = {product: 0 for product in self.products}
        self.mean_spread = 0
        self.std_spread = 0
        self.timestamp = 0
        self.profit_threshold = 0.005  # Threshold for closing positions
        
    def run(self, state):

        # Get the order depth for both products
        P_depth = state.order_depths[self.product[0]]
        C_depth = state.order_depths[self.product[1]]

        # Get the bid and ask prices for both products
        bid_price_P = max(order_depth_P.buy_orders.keys())
        ask_price_P = min(order_depth_P.sell_orders.keys())
        bid_price_C = max(order_depth_C.buy_orders.keys())
        ask_price_C = min(order_depth_C.sell_orders.keys())

        # Compute the spread between the products' bid-ask prices
        spread = bid_price_P - ask_price_C
        

        # Compute z-score
        if self.timestamp == 0:
            self.mean_spread = spread
            self.std_spread = 0
        else:
            self.mean_spread = (self.mean_spread * self.timestamp + spread) / (self.timestamp + 1)
            self.std_spread = np.sqrt(((self.timestamp - 1) * self.std_spread**2 + (spread - self.mean_spread)**2) / self.timestamp)
        
        z_score = (spread - self.mean_spread) / self.std_spread

        result = {}

        # Determine if a buy or sell order should be placed for product 1
        
        # If the spread is above the mean plus the threshold, sell product1 and buy product2
        if z_score > 2:
            if self.pos[self.product[0]] < self.pos_limit[self.products[0]]:
                # buy P
                # P_depth.get() trying to get ask_volume, wrong here
                buy_volume = min(P_depth.get(), self.pos_limit[self.product[0]] - self.pos[self.product[0]])
                result[self.products[0]] = [Order(self.products[0], ask_price_P, buy_volume)]
                self.pos[self.products[0]] += buy_volume
                
            if self.pos[self.product[1]] >- self.pos_limit[self.product[1]]:
                # sell product2
                # C_depth.get() trying to get bid_volume, wrong here
                sell_volume = min(C_depth.get(), self.pos_limit[self.products[1]] + self.pos[self.products[1]])
                result[self.products[1]] = [Order(self.products[1], bid_price_C, -sell_volume)]
                self.pos[self.products[1]] -= sell_volume
        elif z_score < -2:
            if self.pos[self.products[0]] > -self.pos_limit[self.products[0]]:
                # Sell stock1
                # P_depth.get() trying to get bid_volume, wrong here
                sell_volume = min(P_depth.get(), self.pos_limit[self.products[0]] + self.pos[self.products[0]])
                result[self.products[0]] = [Order(self.products[0], bid_price_P, -sell_volume)]
                self.pos[self.products[0]] -= sell_volume

            if self.pos[self.products[1]] < self.pos_limit[self.products[1]]:
                # Buy stock2
                # P_depth.get() trying to get ask_volume, wrong here
                buy_volume = min(C_depth.get(), self.pos_limit[self.products[1]] - self.pos[self.products[1]])
                result[self.products[1]] = [Order(self.products[1],ask_price_P, buy_volume)]
                self.pos[self.products[1]] += buy_volume
        elif abs(z_score) < 2: # Determine if any positions need to be closed
            # Close long position in product2
            sell_volume = self.long_pos[self.products[1]]
            result[self.products[1]] = [Order(self.products[1], bid_price_C, -sell_volume)]
            self.pos[self.products[1]] -= sell_volume
            # Close short position in product2
            buy_volume = self.short_pos[self.products[1]]
            result[self.products[1]] = [Order(self.products[1], ask_price_C, buy_volume)]
            self.pos[self.products[1]] += buy_volume

        # Increment timestamp
        self.timestamp += 1

        return result


